In [10]:
from os import listdir
import cv2
import numpy as np
from skimage.util import random_noise
from imutils import grab_contours
import copy
import matplotlib.pyplot as plt

In [25]:
# Load image
image = cv2.imread("character_image/9/9_01.jpg")
resized = cv2.resize(image, (28,28))
# convert picture to gray scale
img_gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
# _,img = cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)
x_flattend = img_gray.reshape(1, 28*28)
# plt.matshow(img_gray)
x_flattend = np.squeeze(x_flattend)
x_flattend = x_flattend/255
inputs  = x_flattend

array([0.02352941, 0.02352941, 0.01568627, 0.01960784, 0.01960784,
       0.01960784, 0.02352941, 0.03529412, 0.03529412, 0.01960784,
       0.04705882, 0.0627451 , 0.06666667, 0.07843137, 0.11372549,
       0.15294118, 0.18823529, 0.29019608, 0.26666667, 0.28627451,
       0.2627451 , 0.16470588, 0.04313725, 0.04705882, 0.05098039,
       0.05490196, 0.0745098 , 0.14901961, 0.02352941, 0.01568627,
       0.01568627, 0.01568627, 0.02352941, 0.02352941, 0.01960784,
       0.02352941, 0.03137255, 0.05098039, 0.09411765, 0.31372549,
       0.61960784, 0.81568627, 0.85490196, 0.85882353, 0.8627451 ,
       0.8627451 , 0.84313725, 0.51764706, 0.18039216, 0.07058824,
       0.09019608, 0.06666667, 0.05098039, 0.03921569, 0.03529412,
       0.03921569, 0.01568627, 0.01568627, 0.01568627, 0.01960784,
       0.02352941, 0.02352941, 0.04313725, 0.04313725, 0.03529412,
       0.27843137, 0.74509804, 0.84313725, 0.80392157, 0.77254902,
       0.78039216, 0.76862745, 0.78431373, 0.76470588, 0.77647

In [29]:

# bias_j = np.random.uniform(0, 1, size=(HIDDEN_NEURONS,1))
# bias_j
# bias_k = np.random.uniform(0, 1, size=(HIDDEN_NEURONS))
# bias_k

array([0.28630237, 0.79918388, 0.18867381, 0.27259284, 0.20952297,
       0.41875261, 0.42839408, 0.56479862, 0.93119416, 0.09277087,
       0.8041568 , 0.41337983, 0.60291477, 0.81707102, 0.10693973,
       0.28475661])

In [12]:
import numpy as np
import math

OUTPUT_NEURONS = 20
INPUT_NEURONS = 28* 28
HIDDEN_NEURONS = 16

def Weight_Initialization():
    # Initializing of the Weights. Random float number between -0.5 to 0.5 for weights.
    np.random.seed(1)
    inputs= np.random.uniform(-0.5, 0.5, size=(INPUT_NEURONS))
    wji= np.random.uniform(-0.5, 0.5, size=(HIDDEN_NEURONS, INPUT_NEURONS))
    wkj = np.random.uniform(-0.5, 0.5, size=(OUTPUT_NEURONS, HIDDEN_NEURONS))
    bias_j = np.random.uniform(0, 1, size=(HIDDEN_NEURONS))
    bias_k = np.random.uniform(0, 1, size=(OUTPUT_NEURONS))
    targets = np.random.uniform(0, 1, size=(OUTPUT_NEURONS))
    return inputs, wji,wkj,bias_j,bias_k,targets

# def Read_Files():
#     # Reading of Segmented Training Files, and Target Files.
def Forward_Input_Hidden(inputs,wji, bias_j):
    # Forward Propagation from Input -> Hidden Layer.
    # Obtain the results at each neuron in the hidden layer.
    # Calculate 𝑁𝑒𝑡𝑗and 𝑂𝑢𝑡𝑗
    
    Netj = np.dot(inputs,wji.T) 
    print("Netj :")
    print(Netj)
    Outj = 1/(1 + math.e**-(Netj + np.transpose(bias_j)))
    print("Outj :")
    print(Outj)
    return Netj,Outj

def Forward_Hidden_Output(Netj,wkj, bias_k):
    # Forward Propagation from Input -> Hidden Layer.
    # Obtain the results at each neuron in the hidden layer.
    # Calculate 𝑁𝑒𝑡kand 𝑂𝑢𝑡k
    Netk = np.dot(Netj,wkj.T) 
    print("Netk :")
    print(Netk)
    Outk = 1/(1 + math.e**-(Netk + np.transpose(bias_k)))
    print("Outk :")
    print(Outk)
    return Netk, Outk


In [13]:
def Check_for_End(Outk, targets, user_set):
    # Check whether the total error is less than the error set by the user or the number of iterations is reached.
    # returns true or false
    def Error_Correction(outs, targets):
        total_error= np.sum(((outs - targets)**2)/2)
        return total_error
    if Error_Correction(Outk, targets)< user_set:
        return True

    else: 
        return False
        

In [14]:
def Weight_Bias_Correction_Output(Outk, targets, Outj):
    # Correction of Weights and Bias between Hidden and Output Layer.
    # Calculate 𝑑𝑤𝑘𝑘𝑗 and 𝑑𝑏𝑘𝑘𝑗
    dwkkj =  np.empty((0, len(Outk)))
    for i in range(len(Outj)):
        temp =(Outk - targets) * Outk*(1 - Outk) * Outj[i]
        dwkkj = np.vstack([dwkkj,temp])
    dbkkj = (Outk - targets) * Outk*(1 - Outk) 
    dwkkj = dwkkj.T
    # print(dwkkj)
    return dwkkj,dbkkj

In [15]:
def Weight_Bias_Correction_Hidden(outj,outk,inputs,target,wkj):
    # Correction of Weights and Bias between Input and Hidden Layer.
    # Calculate 𝑑𝑤𝑗𝑗𝑖 and 𝑑𝑏𝑗𝑗𝑖
    skl = (outk - target) * outk*(1-outk)
    dwjji= np.multiply.outer(outj *(1 - outj) * np.dot(skl,wkj),inputs)
    dbjii = outj *(1 - outj) * np.dot(skl,wkj)

    return dwjji, dbjii

In [16]:
def Weight_Bias_Update(wkj,dwkkj, bias_k, dbkkj, wji, dwjji,bias_j,dbjii ):
    # Saving_Weights_Bias() implemented inside
    # Update Weights and Bias.
    # Calculate 𝑤𝑘𝑘𝑗+ and 𝑏𝑘𝑘𝑗+
    n = 0.5
    wkjj = wkj - n*dwkkj
    bkkj = bias_k - n*dbkkj
    print("wk+")
    print(wkjj)
    print("bias_k+")
    print(bkkj)

    # Calculate 𝑤𝑗𝑗𝑖+ and 𝑏𝑗𝑗𝑖+
    wjji = wji - n *dwjji
    bjji = bias_j - n* dbjii
    print("wj+")
    print(wjji)
    print("bias_j+")
    print(bjji)

In [17]:
# def Saving_Weights_Bias():
#     # Save 𝑤𝑘𝑘𝑗 and 𝑏𝑘𝑘𝑗
#     # Save 𝑤𝑗𝑗𝑖 and 𝑏𝑗𝑗𝑖


In [31]:
wji,wkj,bias_j,bias_k,target = Weight_Initialization()
# inputs= np.array([0.2,0.5])
# wji= np.array([[0.1,0.2],[0.3,0.4]])
# wkj= np.array([[0.5,0.6],[0.7,0.8]])
# bias_j = np.array([0.2,0.2])
# bias_k = np.array([0.4,0.4])
# target = np.array([0.2,0.8])
# inputs= np.array([0.5,0.8])
# wji= np.array([[-0.8,0.2],[-0.5,-0.2],[0.4,0.4]])
# wkj= np.array([[-0.3,0.15,0.4],[0.4,-0.5,0.1]])
# bias_j = np.array([0.4,0.2,0.1])
# bias_k = np.array([0.3,0.15])
# target = np.array([0.7,0.4])
netj,outj = Forward_Input_Hidden(inputs, wji, bias_j)
netk,outk = Forward_Hidden_Output(outj, wkj, bias_k)

dwkkj,dbkkj = Weight_Bias_Correction_Output(outk,target, outj)

dwjji, dbjii = Weight_Bias_Correction_Hidden(outj,outk,inputs,target,wkj)

Weight_Bias_Update(wkj,dwkkj, bias_k, dbkkj, wji, dwjji,bias_j,dbjii)

# Error_Correction(outk, target)

Netj :
[-0.24 -0.41  0.52]
Outj :
[0.53991488 0.44769209 0.65021855]
Netk :
[0.16526677 0.05714176]
Outk :
[0.61426285 0.55160107]
wk+
[[-0.29451584  0.15454741  0.40660456]
 [ 0.38987751 -0.50839347  0.08780951]]
bias_k+
[0.31015745 0.1312517 ]
wj+
[[-0.80130992  0.19790413]
 [-0.49865269 -0.1978443 ]
 [ 0.40024883  0.40039813]]
bias_j+
[0.39738016 0.20269462 0.10049766]
